**Daily FX Rates Extraction for currency pairs:**
- EUR/USD
- GBP/USD
- USD/JPY
- USD/CHF
- USD/BRL

In [0]:
from pathlib import Path
import requests
from datetime import datetime
import pytz
import pyspark.sql
from pyspark.sql import SparkSession
import sys

# Add src to path for imports
sys.path.insert(0, '/home/user/market_risk')
from src.utils.config_loader import quick_load

spark = SparkSession.builder.getOrCreate()

POLYGON_API_KEY = dbutils.secrets.get(scope="guidotognini", key="polygon_api_key")

# Load configuration
cfg = quick_load()
CURRENCY_PAIRS = cfg['currency_pairs']

TZ = "Europe/Madrid"
current_date = datetime.now(pytz.timezone(TZ)).strftime("%Y-%m-%d")

FROM_DATE = spark.sql("SELECT MAX(date) FROM tabular.dataexpert.silver_fx_daily_rates").collect()[0][0]

TO_DATE = current_date

output_dir = Path("/Volumes/tabular/dataexpert/guidotognini/market_risk/raw_fx_daily_rates")

for pair in CURRENCY_PAIRS:
    url = f"https://api.polygon.io/v2/aggs/ticker/C:{pair}/range/1/day/{FROM_DATE}/{TO_DATE}?apiKey={POLYGON_API_KEY}"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    (output_dir / f"{pair}_{FROM_DATE}_{TO_DATE}.json").write_text(r.text)
    print(f"Downloaded {pair}")